# Modelos lineales regularizados


In [1]:
R.version.string

[1] "R version 4.1.3 (2022-03-10)"

Revisamos los paquetes que se encuentran instalados con los que queremos instalar

In [2]:
packages <- c('ISLR',
              "glmnet",
              'ggplot2',
              'leaps')

not_installed <- !packages %in% installed.packages()
if (any(not_installed)) install.packages(packages[not_installed])

La librería ISLR es provista por el libro que lleva ese nombre y contiene bases de datos de ejemplo para trabajar.

En este caso vamos a predecir el salario de los jugadores de beisbol con modelos lineales

In [3]:
library(ISLR)
fix(Hitters)

In [4]:
names(Hitters)

[1] "AtBat"     "Hits"      "HmRun"     "Runs"      "RBI"       "Walks"    
 [7] "Years"     "CAtBat"    "CHits"     "CHmRun"    "CRuns"     "CRBI"     
[13] "CWalks"    "League"    "Division"  "PutOuts"   "Assists"   "Errors"   
[19] "Salary"    "NewLeague"

In [5]:
dim(Hitters)

[1] 322  20

In [6]:
Hitters_ <- na.omit(Hitters)

## Mínimos cuadrados ordinarios

In [9]:
ols <- lm(Salary ~. ,data= Hitters_)

In [10]:
summary(ols)


Call:
lm(formula = Salary ~ ., data = Hitters_)

Residuals:
    Min      1Q  Median      3Q     Max 
-907.62 -178.35  -31.11  139.09 1877.04 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  163.10359   90.77854   1.797 0.073622 .  
AtBat         -1.97987    0.63398  -3.123 0.002008 ** 
Hits           7.50077    2.37753   3.155 0.001808 ** 
HmRun          4.33088    6.20145   0.698 0.485616    
Runs          -2.37621    2.98076  -0.797 0.426122    
RBI           -1.04496    2.60088  -0.402 0.688204    
Walks          6.23129    1.82850   3.408 0.000766 ***
Years         -3.48905   12.41219  -0.281 0.778874    
CAtBat        -0.17134    0.13524  -1.267 0.206380    
CHits          0.13399    0.67455   0.199 0.842713    
CHmRun        -0.17286    1.61724  -0.107 0.914967    
CRuns          1.45430    0.75046   1.938 0.053795 .  
CRBI           0.80771    0.69262   1.166 0.244691    
CWalks        -0.81157    0.32808  -2.474 0.014057 *  
LeagueN       62.

Este sería el modelo conocido que tabajamos, estimamos con todos los datos y consideramos las variables que son significativas. Pero para predicción, la idea es testear con datos que están por fuera de la muestra. Para esto vamos a partir los datos que tenemos en dos y tomamos una parte de etnrenamiento y la otra de test. Esto se puede hacer con la función *sample*

In [94]:
set.seed(45) #elegimos una semilla para que salgo siempre el mismo resultado
train <- sample(1: nrow(Hitters_), round(nrow(Hitters_)*0.5 ))
test = (-train)
y <- Hitters_['Salary']
X <- Hitters_[,-which(names(Hitters_) == 'Salary')]
y.train <- y[train,] 
y.test <-  y[test,]
X.train <- X[train,]
X.test <-  X[test,]


In [95]:
c(nrow(X.train),length(y.train))


[1] 132 132

In [96]:
c(nrow(X.test),length(y.test))

[1] 131 131

In [97]:
mse <- function(modelo, x_test, y_test){
    pred <- predict(modelo, x_test)
    return(sqrt(sum((pred-y_test)^2)))
    }
    

Ahora calculamos el modelo ols pero sobre la base de entrenamiento, para probar su capacidad predictiva

In [98]:
ols <- lm(Salary ~. , data = Hitters_[train,])

c(mse(ols, X.train,y.train),mse(ols, X.test, y.test))

[1] 2789.718 4652.857

In [64]:
library(leaps)
regfit.full <- regsubsets(Salary ~. , Hitters)
summary(regfit.full)

Subset selection object
Call: regsubsets.formula(Salary ~ ., Hitters)
19 Variables  (and intercept)
           Forced in Forced out
AtBat          FALSE      FALSE
Hits           FALSE      FALSE
HmRun          FALSE      FALSE
Runs           FALSE      FALSE
RBI            FALSE      FALSE
Walks          FALSE      FALSE
Years          FALSE      FALSE
CAtBat         FALSE      FALSE
CHits          FALSE      FALSE
CHmRun         FALSE      FALSE
CRuns          FALSE      FALSE
CRBI           FALSE      FALSE
CWalks         FALSE      FALSE
LeagueN        FALSE      FALSE
DivisionW      FALSE      FALSE
PutOuts        FALSE      FALSE
Assists        FALSE      FALSE
Errors         FALSE      FALSE
NewLeagueN     FALSE      FALSE
1 subsets of each size up to 8
Selection Algorithm: exhaustive
         AtBat Hits HmRun Runs RBI Walks Years CAtBat CHits CHmRun CRuns CRBI
1  ( 1 ) " "   " "  " "   " "  " " " "   " "   " "    " "   " "    " "   "*" 
2  ( 1 ) " "   "*"  " "   " "  " " " "   

In [65]:
(mean((predict(ols)-Hitters$Salary)^2))

Warning message in predict(ols) - Hitters$Salary:
"longitud de objeto mayor no es múltiplo de la longitud de uno menor"


[1] NA

In [70]:
x <- model.matrix(Salary ~., Hitters_)[,-1]
y <- Hitters_$Salary

In [71]:
require(glmnet)
grid <- 10^seq(10, -2, length = 100)
ridge.mod <- glmnet(x,y, alpha = 0, lambda = grid)

In [72]:
dim(coef(ridge.mod))

[1]  20 100

In [73]:
coef(ridge.mod)[,50]

(Intercept)         AtBat          Hits         HmRun          Runs 
407.356050200   0.036957182   0.138180344   0.524629976   0.230701523 
          RBI         Walks         Years        CAtBat         CHits 
  0.239841459   0.289618741   1.107702929   0.003131815   0.011653637 
       CHmRun         CRuns          CRBI        CWalks       LeagueN 
  0.087545670   0.023379882   0.024138320   0.025015421   0.085028114 
    DivisionW       PutOuts       Assists        Errors    NewLeagueN 
 -6.215440973   0.016482577   0.002612988  -0.020502690   0.301433531

In [74]:
predict(ridge.mod, s = 50, type= "coefficients")[1:20,]

(Intercept)         AtBat          Hits         HmRun          Runs 
 4.876610e+01 -3.580999e-01  1.969359e+00 -1.278248e+00  1.145892e+00 
          RBI         Walks         Years        CAtBat         CHits 
 8.038292e-01  2.716186e+00 -6.218319e+00  5.447837e-03  1.064895e-01 
       CHmRun         CRuns          CRBI        CWalks       LeagueN 
 6.244860e-01  2.214985e-01  2.186914e-01 -1.500245e-01  4.592589e+01 
    DivisionW       PutOuts       Assists        Errors    NewLeagueN 
-1.182011e+02  2.502322e-01  1.215665e-01 -3.278600e+00 -9.496680e+00

In [75]:
predict(ridge.mod)

ERROR: Error in predict.glmnet(ridge.mod): You need to supply a value for 'newx'


In [82]:
library(MASS)
head(Boston)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.00632,18,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
2,0.02731,0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
3,0.02729,0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
6,0.02985,0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7


In [84]:
ols <- lm(medv~., data = Boston)
summary(ols)


Call:
lm(formula = medv ~ ., data = Boston)

Residuals:
    Min      1Q  Median      3Q     Max 
-15.595  -2.730  -0.518   1.777  26.199 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.646e+01  5.103e+00   7.144 3.28e-12 ***
crim        -1.080e-01  3.286e-02  -3.287 0.001087 ** 
zn           4.642e-02  1.373e-02   3.382 0.000778 ***
indus        2.056e-02  6.150e-02   0.334 0.738288    
chas         2.687e+00  8.616e-01   3.118 0.001925 ** 
nox         -1.777e+01  3.820e+00  -4.651 4.25e-06 ***
rm           3.810e+00  4.179e-01   9.116  < 2e-16 ***
age          6.922e-04  1.321e-02   0.052 0.958229    
dis         -1.476e+00  1.995e-01  -7.398 6.01e-13 ***
rad          3.060e-01  6.635e-02   4.613 5.07e-06 ***
tax         -1.233e-02  3.760e-03  -3.280 0.001112 ** 
ptratio     -9.527e-01  1.308e-01  -7.283 1.31e-12 ***
black        9.312e-03  2.686e-03   3.467 0.000573 ***
lstat       -5.248e-01  5.072e-02 -10.347  < 2e-16 ***
---
Signif. codes:  0

In [92]:
set.seed(45) #elegimos una semilla para que salgo siempre el mismo resultado
train <- sample(1: nrow(Boston), round(nrow(Boston)*0.5 ))
test = (-train)
y <- Boston['medv']
X <- Boston[,-which(names(Boston) == 'medv')]
y.train <- y[train,] 
y.test <-  y[test,]
X.train <- X[train,]
X.test <-  X[test,]

In [93]:
ols <- lm(medv~. , data = Boston[train,])
c(mse(ols, X.train, y.train),mse(ols, X.test, y.test))

[1] 69.72997 80.06199